## Define Dynamic Parameters

In [1]:
file_no = 0
my_seed = 32

## Import Packages

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.model_selection import  train_test_split
import tensorflow as tf
import ntpath

tf.random.set_seed(my_seed)
np.random.seed(my_seed)

## Import CSVs

In [3]:
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '\\..\\..\\Datasets\\processed\\classification_10d'
print(path)

csvfiles = glob.glob(path + "/*.csv")
dfs = []

for file in csvfiles:
    df = pd.read_csv(file)
    df.name = (ntpath.basename(file)).split(".",1)[0]
    dfs.append(df)

df = dfs[file_no]
print("Stock: ")
print(df.name)
df_name = df.name


C:\FYP - Luke Bezzina\Code\mlpLearning\..\..\Datasets\processed\classification_10d
Stock: 
AAL


# Encoding Out Variable

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# y = np.array(ct.fit_transform(y))
# print(y)

price_classification = ['Positive', 'Neutral', 'Negative']

price_direction = df.PriceDirection
one_hot_dummies = pd.get_dummies(price_direction)
one_hot_dummies = one_hot_dummies.reindex\
    (columns=price_classification, fill_value=0)

df = df.drop(columns=['PriceDirection'])
df = pd.concat([df, one_hot_dummies], axis=1)

print(df)


           Date  d10_pd  d9_pd  d8_pd  d7_pd  d6_pd  d5_pd  d4_pd  d3_pd  \
0    2017-01-19    0.40  -0.41  -0.09   0.78   2.18   2.34   1.80   1.35   
1    2017-01-20   -0.81  -0.49   0.38   1.78   1.94   1.40   0.95   0.05   
2    2017-01-23    0.32   1.19   2.59   2.75   2.21   1.76   0.86   1.75   
3    2017-01-24    0.87   2.27   2.43   1.89   1.44   0.54   1.43   1.05   
4    2017-01-25    1.40   1.56   1.02   0.57  -0.33   0.56   0.18   0.92   
..          ...     ...    ...    ...    ...    ...    ...    ...    ...   
991  2020-12-24    0.83  -0.11  -0.46  -0.15  -0.30  -0.36  -0.65  -1.06   
992  2020-12-28   -0.94  -1.29  -0.98  -1.13  -1.19  -1.48  -1.89  -2.51   
993  2020-12-29   -0.35  -0.04  -0.19  -0.25  -0.54  -0.95  -1.57  -1.16   
994  2020-12-30    0.31   0.16   0.10  -0.19  -0.60  -1.22  -0.81  -1.04   
995  2020-12-31   -0.15  -0.21  -0.50  -0.91  -1.53  -1.12  -1.35  -0.95   

     d2_pd  d1_pd     Trend  VolumeTrend  Volatility  PreviousClose  Positive  \
0     

# Splitting dataset in dependent and independent variables

In [5]:
x = np.array(df.iloc[:, 1:-3].values, dtype=np.float)
y = np.array(df.iloc[:, -3:].values)

# Splitting dataset (training + testing)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False, random_state=32)

print("Train - Independent Variables")
print(x_train)
print("Test - Independent Variables")
print(x_test)
print("Train - Dependent Variables")
print(y_train)
print("Test - Dependent Variables")
print(y_test)


Train - Independent Variables
[[ 4.00000000e-01 -4.10000000e-01 -9.00000000e-02 ... -3.01256670e-02
   8.91697258e-01  4.76400000e+01]
 [-8.10000000e-01 -4.90000000e-01  3.80000000e-01 ... -5.45020542e-02
   8.68665643e-01  4.72600000e+01]
 [ 3.20000000e-01  1.19000000e+00  2.59000000e+00 ... -3.55820268e-02
   7.28497769e-01  4.80000000e+01]
 ...
 [ 4.10000000e-01  3.00000000e-02  3.00000000e-02 ... -1.62120342e-02
   4.15452765e-01  1.25600000e+01]
 [-3.80000000e-01 -3.80000000e-01  0.00000000e+00 ...  5.26609189e-02
   6.18902254e-01  1.30400000e+01]
 [ 0.00000000e+00  3.80000000e-01  1.00000000e-02 ...  8.84357847e-02
   7.36955901e-01  1.30300000e+01]]
Test - Independent Variables
[[ 3.80000000e-01  1.00000000e-02 -2.10000000e-01 ...  1.09585659e-01
   9.69579806e-01  1.40000000e+01]
 [-3.70000000e-01 -5.90000000e-01 -6.50000000e-01 ...  1.56196285e-01
   1.08175274e+00  1.37300000e+01]
 [-2.20000000e-01 -2.80000000e-01 -3.20000000e-01 ...  1.46665876e-01
   1.10174634e+00  1.3540

# Building ANN with MLP

In [7]:
ann = tf.keras.models.Sequential()
# hidden layers
ann.add(tf.keras.layers.Dense(units=9, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam()
ann.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

ann.fit(x_train, y_train, batch_size = 32, epochs = 200)

weights = ann.get_weights()

Epoch 1/200
28/28 [==============================] - 1s 1ms/step - loss: 0.9451 - accuracy: 0.2213
Epoch 2/200
28/28 [==============================] - 0s 968us/step - loss: 0.7356 - accuracy: 0.4204
Epoch 3/200
28/28 [==============================] - 0s 1ms/step - loss: 0.6708 - accuracy: 0.4189
Epoch 4/200
28/28 [==============================] - 0s 924us/step - loss: 0.6311 - accuracy: 0.4421
Epoch 5/200
28/28 [==============================] - 0s 930us/step - loss: 0.6001 - accuracy: 0.4376
Epoch 6/200
28/28 [==============================] - 0s 920us/step - loss: 0.5852 - accuracy: 0.4868

# Predicting Test Set Results

In [8]:
y_pred = ann.predict(x_test)
# boolean result of whether predicted probability is larger than 0.5
# result > 0.5 = 1, result <= 0.5 = 0
y_pred_binary = []
for i in y_pred:
    max = 0
    elem = 0
    for iter, x in enumerate(i):
        if x > max:
            max = x
            elem = iter
    rec = [False, False, False]
    rec[elem] = True
    y_pred_binary.append(rec)

y_pred_binary = np.array(y_pred_binary)

# Obtaining Confusion Matrix and Accuracy Score for predictions

In [9]:
%%capture cap

# To evaluate accuracy, a vector of labels is needed
# np.argmax(x, axis=1) - to output correct labels
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# Confusion Matrix: Columns = 0, 1 (Predicted Label)
# Confusion Matrix: Rows = 0, 1 (Actual Label)
cm = multilabel_confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred_binary, axis=1))
print(cm)

print()
print("Accuracy Score: ")
print(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_binary, axis=1)))



# Output Results

In [10]:
with open(ROOT_DIR+'\\classification_results\\'+df_name + '_' + str(my_seed) +'new.txt', 'w') as f:
    f.write(cap.stdout)

